## **Download Woodscape**

In [16]:
! pip install gdown scikit-learn pyyaml

  Using cached PyYAML-6.0.2-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (2.1 kB)
Using cached PyYAML-6.0.2-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (767 kB)


In [2]:
import zipfile
import os
import gdown

def download_from_google_drive(output: str, file_id: str):
    url = f"https://drive.google.com/uc?id={file_id}"
    output_zip = f"{output}/archive.zip"

    gdown.download(url, output_zip, quiet=False)

    with zipfile.ZipFile(output_zip, 'r') as zip_ref:
        zip_ref.extractall(output)
    
    os.remove(output_zip)

In [1]:
import os
os.chdir("../../../..") 

In [2]:
WOODSCAPE_PATH = "src/datasets/WoodScape"
IMAGES_FILE_ID = "1xQ5J4huNmyK9WPoipHTnuZ7lw_J0xhvL"
ANNOTATIONS_FILE_ID = "1k9q1k8rh6hghSPFdmxmkZUd_Mip6pt6F"

download_from_google_drive(WOODSCAPE_PATH, IMAGES_FILE_ID)
download_from_google_drive(WOODSCAPE_PATH, ANNOTATIONS_FILE_ID)

NameError: name 'download_from_google_drive' is not defined

In [3]:
WOODSCAPE_PATH = "src/datasets/WoodScape"

IMAGES_PATH = f"{WOODSCAPE_PATH}/rgb_images"
ANNOTATIONS_PATH = f"{WOODSCAPE_PATH}/instance_annotations"
CLASSINFO_PATH = f"{WOODSCAPE_PATH}/class_info.json"

## **Convert Woodscape annotations to Ultralytics**

In [4]:
YOLO_ANNOTATIONS_PATH = f"{WOODSCAPE_PATH}/yolo_annotations"

In [ ]:
from src.models.train.convert import convert_Woodscape_to_Ultralytics
import shutil

convert_Woodscape_to_Ultralytics(ANNOTATIONS_PATH, YOLO_ANNOTATIONS_PATH, CLASSINFO_PATH)
shutil.rmtree(ANNOTATIONS_PATH)

AttributeError: module 'os' has no attribute 'rmtree'

## **Split dataset**

In [5]:
TRAIN_DATASET = f"{WOODSCAPE_PATH}/train_dataset"

In [12]:
from src.models.train.split_dataset import split_dataset
split_dataset(IMAGES_PATH, YOLO_ANNOTATIONS_PATH, TRAIN_DATASET)

In [18]:
shutil.rmtree(YOLO_ANNOTATIONS_PATH)
shutil.rmtree(IMAGES_PATH)

## **Create YAML file**

In [ ]:
import yaml
import json

with open(CLASSINFO_PATH, "r", encoding="utf-8") as f:
    CLASSES = json.load(f)["classes"]

data = {
    "path": TRAIN_DATASET,
    "train": "images/train",
    "val": "images/val",
    "test": "images/test",
    "nc": len(CLASSES),
    "names": CLASSES,

    "mask_ratio": 2,
    "overlap_mask": True,
    "preprocessing": {
        "auto_augment": "none",
        "erasing": 0.0,
        "mosaic": 0.5,
        "copy_paste": 0.0
    }
}

with open(f"{WOODSCAPE_PATH}/woodscape.yaml", "w") as file:
    yaml.dump(data, file, default_flow_style=False)
